<a href="https://colab.research.google.com/github/sipocz/Blood/blob/main/lazyregressor_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install lazypredict


In [2]:
import pandas as pd

from sklearn.preprocessing import MinMaxScaler

In [3]:
fname_url="https://github.com/sipocz/Blood/raw/b5fc44c487b0a712d00865014203349fb8690257/orig/train.zip"
fname=fname_url.split("/")[-1]

In [4]:
!wget $fname_url


--2021-12-06 19:14:31--  https://github.com/sipocz/Blood/raw/b5fc44c487b0a712d00865014203349fb8690257/orig/train.zip
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/sipocz/Blood/b5fc44c487b0a712d00865014203349fb8690257/orig/train.zip [following]
--2021-12-06 19:14:31--  https://raw.githubusercontent.com/sipocz/Blood/b5fc44c487b0a712d00865014203349fb8690257/orig/train.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16979362 (16M) [application/zip]
Saving to: ‘train.zip’

train.zip           100%[===================>]  16.19M  --.-KB/s    in 0.09s   

2021-12-06 19:14:32 (175 MB/s) - ‘train.zi

In [5]:
!mkdir "train"
!rm ./train/Train.csv

rm: cannot remove './train/Train.csv': No such file or directory


In [6]:
!unzip $fname -d "./train"
!rm train.zip

Archive:  train.zip
  inflating: ./train/Train.csv       


In [7]:
fname_train="./train/Train.csv"
df=pd.read_csv(fname_train)

In [8]:
df.head(5)

,Reading_ID,absorbance0,absorbance1,absorbance2,absorbance3,absorbance4,absorbance5,absorbance6,absorbance7,absorbance8,absorbance9,absorbance10,absorbance11,absorbance12,absorbance13,absorbance14,absorbance15,absorbance16,absorbance17,absorbance18,absorbance19,absorbance20,absorbance21,absorbance22,absorbance23,absorbance24,absorbance25,absorbance26,absorbance27,absorbance28,absorbance29,absorbance30,absorbance31,absorbance32,absorbance33,absorbance34,absorbance35,absorbance36,absorbance37,absorbance38,...,absorbance135,absorbance136,absorbance137,absorbance138,absorbance139,absorbance140,absorbance141,absorbance142,absorbance143,absorbance144,absorbance145,absorbance146,absorbance147,absorbance148,absorbance149,absorbance150,absorbance151,absorbance152,absorbance153,absorbance154,absorbance155,absorbance156,absorbance157,absorbance158,absorbance159,absorbance160,absorbance161,absorbance162,absorbance163,absorbance164,absorbance165,absorbance166,absorbance167,absorbance168,absorbance169,temperature,humidity,hdl_cholesterol_human,hemoglobin(hgb)_human,cholesterol_ldl_human
0,ID_3SSHI56C,0.479669,0.477423,0.487956,0.491831,0.500516,0.502590,0.511561,0.514639,0.524245,0.536170,0.546407,0.561557,0.568417,0.571877,0.570884,0.569032,0.567476,0.565662,0.561901,0.559722,0.557474,0.554371,0.552386,0.548702,0.544238,0.542579,0.540514,0.538980,0.536650,0.536483,0.535447,0.537577,0.535715,0.536895,0.539589,0.541081,0.544893,0.547765,0.551773,...,1.469838,1.462617,1.445696,1.435586,1.417847,1.404205,1.388861,1.377436,1.364444,1.360373,1.341243,1.339632,1.321471,1.317444,1.311209,1.291677,1.285579,1.285488,1.275784,1.271104,1.264029,1.250779,1.254856,1.255224,1.249623,1.244419,1.244437,1.243933,1.226790,1.234055,1.218660,1.213038,1.198317,1.195735,1.180846,42.51,34.01,ok,ok,ok
1,ID_599OOLZA,0.471537,0.474113,0.479981,0.485528,0.491049,0.497942,0.504760,0.510543,0.522328,0.534423,0.548646,0.558420,0.565449,0.569717,0.570999,0.569969,0.568405,0.566628,0.564101,0.559951,0.556193,0.552271,0.550086,0.546207,0.542366,0.539789,0.537221,0.534336,0.533868,0.533018,0.532227,0.530818,0.532171,0.533658,0.535266,0.538939,0.542399,0.546479,0.550606,...,1.552979,1.541997,1.533186,1.518359,1.498964,1.488043,1.472946,1.465925,1.452647,1.437819,1.423670,1.415103,1.401141,1.403560,1.384169,1.379410,1.374128,1.356969,1.352693,1.342430,1.339714,1.332805,1.336324,1.342537,1.332407,1.326258,1.336874,1.327538,1.311951,1.309399,1.304501,1.323005,1.305992,1.263887,1.262095,44.52,32.09,ok,high,high
2,ID_MVJGPQ75,0.444998,0.458034,0.447386,0.456921,0.463225,0.475983,0.476817,0.481565,0.490010,0.505892,0.518125,0.530362,0.538530,0.543128,0.546287,0.547001,0.547120,0.546351,0.544254,0.542802,0.542207,0.539779,0.536417,0.533380,0.531117,0.529093,0.526101,0.524599,0.522952,0.521551,0.521149,0.520478,0.521432,0.521473,0.523567,0.525816,0.527889,0.530697,0.533416,...,1.516723,1.502255,1.489132,1.483308,1.461028,1.453174,1.450412,1.437784,1.422148,1.415880,1.404698,1.388143,1.397241,1.385680,1.376355,1.357758,1.354854,1.345476,1.333810,1.332739,1.335550,1.326775,1.336862,1.316860,1.328051,1.328641,1.323526,1.314124,1.298936,1.289122,1.325059,1.271115,1.337119,1.289877,1.345229,45.77,24.80,ok,ok,high
3,ID_CK6RF8YV,0.513434,0.513303,0.522609,0.521068,0.523146,0.530132,0.539517,0.546364,0.552414,0.565502,0.581143,0.594354,0.599457,0.604529,0.605267,0.606276,0.604895,0.603716,0.600683,0.598087,0.594303,0.589403,0.585883,0.581369,0.578962,0.575181,0.573274,0.570471,0.568241,0.565671,0.564579,0.563724,0.561978,0.562744,0.563455,0.565163,0.566505,0.569239,0.572075,...,1.442957,1.423349,1.413718,1.403112,1.393964,1.375741,1.369549,1.354179,1.344562,1.333491,1.325002,1.321572,1.305561,1.292637,1.287971,1.283460,1.278300,1.268486,1.268407,1.263479,1.252612,1.254306,1.247635,1.242321,1.247859,1.246749,1.249920,1.265223,1.264013,1.285252,1.298422,1.299873,1.311157,1.303259,1.349833,45.84,36.93,low,ok,high
4,ID_82N6QE6I,0.510485,0.519359,0.524225,0.528419,0.535273,0.545342,0.550314,0.55

In [107]:
def one_hot_encode_df(df,str_index):
    from sklearn.preprocessing import OneHotEncoder
    indf=df.copy(True)
    one_hot=OneHotEncoder()
    col=indf[str_index].values
    col=col.reshape(-1,1)
    one_hot.fit(col)
    out=one_hot.transform(col).toarray()
    codes=one_hot.get_feature_names()
    print(codes)
    for ind,code in enumerate(codes):
        print("Itt:",ind,codes[ind])
        indf[str_index+"_"+code]=out[:,ind]
    return(indf,one_hot)

In [108]:
df,hdl_onehot=one_hot_encode_df(df,"hdl_cholesterol_human")
df,hemo_onehot=one_hot_encode_df(df,"hemoglobin(hgb)_human")
df,col_onehot=one_hot_encode_df(df,"cholesterol_ldl_human")

['x0_high' 'x0_low' 'x0_ok']
Itt: 0 x0_high
Itt: 1 x0_low
Itt: 2 x0_ok
['x0_high' 'x0_low' 'x0_ok']
Itt: 0 x0_high
Itt: 1 x0_low
Itt: 2 x0_ok
['x0_high' 'x0_low' 'x0_ok']
Itt: 0 x0_high
Itt: 1 x0_low
Itt: 2 x0_ok


In [109]:
df.tail()

,Reading_ID,absorbance0,absorbance1,absorbance2,absorbance3,absorbance4,absorbance5,absorbance6,absorbance7,absorbance8,absorbance9,absorbance10,absorbance11,absorbance12,absorbance13,absorbance14,absorbance15,absorbance16,absorbance17,absorbance18,absorbance19,absorbance20,absorbance21,absorbance22,absorbance23,absorbance24,absorbance25,absorbance26,absorbance27,absorbance28,absorbance29,absorbance30,absorbance31,absorbance32,absorbance33,absorbance34,absorbance35,absorbance36,absorbance37,absorbance38,...,absorbance144,absorbance145,absorbance146,absorbance147,absorbance148,absorbance149,absorbance150,absorbance151,absorbance152,absorbance153,absorbance154,absorbance155,absorbance156,absorbance157,absorbance158,absorbance159,absorbance160,absorbance161,absorbance162,absorbance163,absorbance164,absorbance165,absorbance166,absorbance167,absorbance168,absorbance169,temperature,humidity,hdl_cholesterol_human,hemoglobin(hgb)_human,cholesterol_ldl_human,hdl_cholesterol_human_x0_high,hdl_cholesterol_human_x0_low,hdl_cholesterol_human_x0_ok,hemoglobin(hgb)_human_x0_high,hemoglobin(hgb)_human_x0_low,hemoglobin(hgb)_human_x0_ok,cholesterol_ldl_human_x0_high,cholesterol_ldl_human_x0_low,cholesterol_ldl_human_x0_ok
13135,ID_NGPC0DA3,0.48,0.48,0.49,0.50,0.50,0.51,0.51,0.52,0.53,0.54,0.55,0.56,0.57,0.57,0.58,0.57,0.57,0.57,0.57,0.56,0.56,0.56,0.55,0.55,0.55,0.55,0.54,0.54,0.54,0.54,0.54,0.54,0.54,0.54,0.54,0.54,0.55,0.55,0.56,...,1.30,1.29,1.30,1.28,1.28,1.27,1.26,1.26,1.25,1.25,1.23,1.23,1.24,1.23,1.22,1.22,1.22,1.22,1.21,1.21,1.20,1.19,1.18,1.21,1.19,1.19,34.79,36.93,high,ok,ok,1.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,1.00
13136,ID_XRBUD5U8,0.53,0.53,0.53,0.53,0.54,0.55,0.55,0.56,0.57,0.58,0.60,0.60,0.61,0.61,0.62,0.62,0.62,0.62,0.61,0.60,0.60,0.60,0.59,0.59,0.59,0.58,0.58,0.58,0.58,0.58,0.58,0.58,0.58,0.58,0.58,0.58,0.59,0.59,0.60,...,1.48,1.46,1.44,1.45,1.43,1.42,1.42,1.41,1.42,1.41,1.39,1.38,1.38,1.38,1.38,1.38,1.38,1.36,1.38,1.38,1.37,1.39,1.38,1.37,1.40,1.39,43.12,19.14,ok,ok,ok,0.00,0.00,1.00,0.00,0.00,1.00,0.00,0.00,1.00
13137,ID_2M9L5NV2,0.51,0.52,0.52,0.53,0.54,0.54,0.55,0.56,0.57,0.58,0.60,0.61,0.62,0.62,0.63,0.63,0.62,0.62,0.62,0.62,0.61,0.61,0.61,0.60,0.60,0.60,0.60,0.59,0.59,0.59,0.59,0.59,0.59,0.59,0.59,0.59,0.59,0.60,0.60,...,1.45,1.43,1.43,1.42,1.41,1.42,1.40,1.40,1.39,1.38,1.38,1.36,1.36,1.37,1.37,1.37,1.36,1.37,1.36,1.39,1.39,1.40,1.38,1.43,1.45,1.38,42.48,43.41,ok,ok,ok,0.00,0.00,1.00,0.00,0.00,1.00,0.00,0.00,1.00
13138,ID_C5V5SD2D,0.46,0.47,0.47,0.47,0.48,0.49,0.49,0.50,0.51,0.52,0.53,0.54,0.55,0.56,0.56,0.56,0.56,0.55,0.55,0.55,0.55,0.55,0.55,0.54,0.54,0.54,0.53,0.53,0.53,0.53,0.53,0.53,0.53,0.53,0.53,0.53,0.54,0.54,0.54,...,1.29,1.29,1.28,1.27,1.26,1.26,1.25,1.25,1.24,1.23,1.22,1.22,1.22,1.23,1.22,1.21,1.21,1.21,1.22,1.21,1.19,1.18,1.17,1.17,1.15,1.21,41.86,35.10,ok,ok,ok,0.00,0.00,1.00,0.00,0.00,1.00,0.00,0.00,1.00
13139,ID_9R5ZR9NX,0.50,0.50,0.51,0.51,0.52,0.52,0.53,0.54,0.55,0.56,0.57,0.59,0.59,0.59,0.60,0.60,0.60,0.60,0.59,0.59,0.58,0.58,0.58,0.57,0.57,0.57,0.57,0.57,0.56,0.56,0.56,0.56,0.56,0.57,0.57,0.57,0.57,0.58,0.58,...,1.47,1.46,1.45,1.43,1.43,1.42,1.41,1.41,1.40,1.39,1.38,1.39,1.38,1.37,1.36,1.37,1.37,1.37,1.37,1.38,1.39,1.38,1.40,1.44,1.35,1.42,42.91,26.30,high,ok,high,1.00,0.00,0.00,0.00,0.00,1.00,1.00,0.00,0.00


In [111]:

del df["hdl_cholesterol_human"]
del df["hemoglobin(hgb)_human"]
del df["cholesterol_ldl_human"]




In [112]:
from lazypredict.Supervised import LazyRegressor
from lazypredict.Supervised import LazyClassifier

from sklearn import datasets
from sklearn.utils import shuffle
import numpy as np


In [116]:
col_names=df.columns[-9:]

In [118]:
col_names[0]

'hdl_cholesterol_human_x0_high'

In [119]:
X=df.iloc[:,1:-9]
y=df.loc[:,col_names[0]]

0       0.00
1       0.00
2       0.00
3       0.00
4       0.00
        ... 
13135   1.00
13136   0.00
13137   0.00
13138   0.00
13139   1.00
Name: hdl_cholesterol_human_x0_high, Length: 13140, dtype: float64

In [122]:
X

,absorbance0,absorbance1,absorbance2,absorbance3,absorbance4,absorbance5,absorbance6,absorbance7,absorbance8,absorbance9,absorbance10,absorbance11,absorbance12,absorbance13,absorbance14,absorbance15,absorbance16,absorbance17,absorbance18,absorbance19,absorbance20,absorbance21,absorbance22,absorbance23,absorbance24,absorbance25,absorbance26,absorbance27,absorbance28,absorbance29,absorbance30,absorbance31,absorbance32,absorbance33,absorbance34,absorbance35,absorbance36,absorbance37,absorbance38,absorbance39,...,absorbance132,absorbance133,absorbance134,absorbance135,absorbance136,absorbance137,absorbance138,absorbance139,absorbance140,absorbance141,absorbance142,absorbance143,absorbance144,absorbance145,absorbance146,absorbance147,absorbance148,absorbance149,absorbance150,absorbance151,absorbance152,absorbance153,absorbance154,absorbance155,absorbance156,absorbance157,absorbance158,absorbance159,absorbance160,absorbance161,absorbance162,absorbance163,absorbance164,absorbance165,absorbance166,absorbance167,absorbance168,absorbance169,temperature,humidity
0,0.48,0.48,0.49,0.49,0.50,0.50,0.51,0.51,0.52,0.54,0.55,0.56,0.57,0.57,0.57,0.57,0.57,0.57,0.56,0.56,0.56,0.55,0.55,0.55,0.54,0.54,0.54,0.54,0.54,0.54,0.54,0.54,0.54,0.54,0.54,0.54,0.54,0.55,0.55,0.55,...,1.52,1.51,1.49,1.47,1.46,1.45,1.44,1.42,1.40,1.39,1.38,1.36,1.36,1.34,1.34,1.32,1.32,1.31,1.29,1.29,1.29,1.28,1.27,1.26,1.25,1.25,1.26,1.25,1.24,1.24,1.24,1.23,1.23,1.22,1.21,1.20,1.20,1.18,42.51,34.01
1,0.47,0.47,0.48,0.49,0.49,0.50,0.50,0.51,0.52,0.53,0.55,0.56,0.57,0.57,0.57,0.57,0.57,0.57,0.56,0.56,0.56,0.55,0.55,0.55,0.54,0.54,0.54,0.53,0.53,0.53,0.53,0.53,0.53,0.53,0.54,0.54,0.54,0.55,0.55,0.55,...,1.62,1.59,1.58,1.55,1.54,1.53,1.52,1.50,1.49,1.47,1.47,1.45,1.44,1.42,1.42,1.40,1.40,1.38,1.38,1.37,1.36,1.35,1.34,1.34,1.33,1.34,1.34,1.33,1.33,1.34,1.33,1.31,1.31,1.30,1.32,1.31,1.26,1.26,44.52,32.09
2,0.44,0.46,0.45,0.46,0.46,0.48,0.48,0.48,0.49,0.51,0.52,0.53,0.54,0.54,0.55,0.55,0.55,0.55,0.54,0.54,0.54,0.54,0.54,0.53,0.53,0.53,0.53,0.52,0.52,0.52,0.52,0.52,0.52,0.52,0.52,0.53,0.53,0.53,0.53,0.54,...,1.54,1.54,1.53,1.52,1.50,1.49,1.48,1.46,1.45,1.45,1.44,1.42,1.42,1.40,1.39,1.40,1.39,1.38,1.36,1.35,1.35,1.33,1.33,1.34,1.33,1.34,1.32,1.33,1.33,1.32,1.31,1.30,1.29,1.33,1.27,1.34,1.29,1.35,45.77,24.80
3,0.51,0.51,0.52,0.52,0.52,0.53,0.54,0.55,0.55,0.57,0.58,0.59,0.60,0.60,0.61,0.61,0.60,0.60,0.60,0.60,0.59,0.59,0.59,0.58,0.58,0.58,0.57,0.57,0.57,0.57,0.56,0.56,0.56,0.56,0.56,0.57,0.57,0.57,0.57,0.58,...,1.48,1.47,1.46,1.44,1.42,1.41,1.40,1.39,1.38,1.37,1.35,1.34,1.33,1.33,1.32,1.31,1.29,1.29,1.28,1.28,1.27,1.27,1.26,1.25,1.25,1.25,1.24,1.25,1.25,1.25,1.27,1.26,1.29,1.30,1.30,1.31,1.30,1.35,45.84,36.93
4,0.51,0.52,0.52,0.53,0.54,0.55,0.55,0.56,0.57,0.58,0.59,0.60,0.61,0.61,0.62,0.62,0.62,0.62,0.62,0.62,0.61,0.61,0.61,0.61,0.60,0.60,0.60,0.60,0.60,0.59,0.59,0.59,0.60,0.60,0.60,0.60,0.60,0.61,0.61,0.61,...,1.46,1.46,1.45,1.43,1.43,1.42,1.41,1.41,1.40,1.39,1.38,1.37,1.36,1.35,1.35,1.35,1.33,1.33,1.32,1.32,1.32,1.30,1.30,1.29,1.30,1.29,1.29,1.29,1.29,1.29,1.30,1.27,1.27,1.27,1.29,1.34,1.32,1.35,38.92,23.88
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13135,0.48,0.48,0.49,0.50,0.50,0.51,0.51,0.52,0.53,0.54,0.55,0.56,0.57,0.57,0.58,0.57,0.57,0.57,0.57,0.56,0.56,0.56,0.55,0.55,0.55,0.55,0.54,0.54,0.54,0.54,0.54,0.54,0.54,0.54,0.54,0.54,0.55,0.55,0.56,0.56,...,1.44,1.43,1.41,1.41,1.39,1.38,1.37,1.36,1.34,1.33,1.33,1.31,1.30,1.29,1.30,1.28,1.28,1.27,1.26,1.26,1.25,1.25,1.23,1.23,1.24,1.23,1.22,1.22,1.22,1.22,1.21,1.21,1.20,1.19,1.18,1.21,1.19,1.19,34.79,36.93
13136,0.53,0.53,0.53,0.53,0.54,0.55,0.55,0.56,0.57,0.58,0.60,0.60,0.61,0.61,0.62,0.62,0.62,0.62,0.61,0.60,0.60,0.60,0.59,0.59,0.59,0.58,0.58,0.58,0.58,0

In [121]:
offset=int(0.9*len(X))
print(offset)

11826


In [95]:
X_train, y_train = X[:offset], y[:offset]
X_test, y_test = X[offset:], y[offset:]

clf = LazyClassifier(verbose=1,ignore_warnings=True, custom_metric=None,predictions=True)
modells,predictions = clf.fit(X_train, X_test, y_train, y_test)


  0%|          | 0/29 [00:03<?, ?it/s]


KeyboardInterrupt: ignored

In [126]:
t1=KNeighborsClassifier()
o=t1.fit(X_train,y_train)

In [133]:
y_train

,hdl_cholesterol_human_x0_ok
0,1.00
1,1.00
2,1.00
3,0.00
4,1.00
...,...
11821,0.00
11822,1.00
11823,1.00
11824,1.00


In [134]:
o.predict

array([0., 1., 0., ..., 1., 1., 1.])

In [147]:
from sklearn.neighbors import  KNeighborsClassifier
my_cl=[]

for i in range(9):

    my_cl.append(  KNeighborsClassifier())
    X=df.iloc[:,1:-9]
    y=df.loc[:,col_names[i]]
    my_cl[i].fit(X,y)



In [148]:
my_cl

[KNeighborsClassifier(),
 KNeighborsClassifier(),
 KNeighborsClassifier(),
 KNeighborsClassifier(),
 KNeighborsClassifier(),
 KNeighborsClassifier(),
 KNeighborsClassifier(),
 KNeighborsClassifier(),
 KNeighborsClassifier()]

In [149]:
my_cl[0].predict(X)

array([0., 0., 0., ..., 0., 0., 1.])

In [150]:
my_cl[1].predict(X)

array([0., 0., 0., ..., 0., 0., 0.])

In [151]:
my_cl[2].predict(X)

array([1., 1., 1., ..., 1., 1., 0.])